<a href="https://colab.research.google.com/github/andrejpetroski/SI_2024/blob/master/Lab_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Decision Trees and Gradient Boosting

## Setting up the Environment

For this laboratory exercise, you will need to install the Anaconda package & environment manager. We will install a minimal distribution, [Miniconda](https://docs.conda.io/projects/miniconda/en/latest/). Choose the adequate distribution for your operating system, download and install it.

Or use the following commands:

### Windows
```shell
curl https://repo.anaconda.com/miniconda/Miniconda3-latest-Windows-x86_64.exe -o miniconda.exe
start /wait "" miniconda.exe /S
del miniconda.exe
```

### Linux
```shell
mkdir -p ~/miniconda3
wget https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh -O ~/miniconda3/miniconda.sh
bash ~/miniconda3/miniconda.sh -b -u -p ~/miniconda3
rm -rf ~/miniconda3/miniconda.sh
```

### macOS

```shell
mkdir -p ~/miniconda3
curl https://repo.anaconda.com/miniconda/Miniconda3-latest-MacOSX-arm64.sh -o ~/miniconda3/miniconda.sh
bash ~/miniconda3/miniconda.sh -b -u -p ~/miniconda3
rm -rf ~/miniconda3/miniconda.sh
```

For both Linux and macOS after installing, initialize your newly-installed Miniconda. The following commands initialize for bash and zsh shells:

```shell
~/miniconda3/bin/conda init bash
~/miniconda3/bin/conda init zsh
```


Once you have installed miniconda, run the following commands to create an environment:
```bash
conda create --name myenv
```

'myenv' is the name of the environment, you can change the name however you want.

When conda asks you to proceed, type y

After successfully creating the environment, activate it with the following command:
```bash
conda activate myenv
```

For more detailed information you can read the [documentation](https://conda.io/projects/conda/en/latest/user-guide/tasks/manage-environments.html#creating-an-environment-with-commands).

Now, once the environment is activated, proceed to install the required libraries.

```bash
pip install numpy pandas scikit-learn xgboost matplotlib seaborn gdown
```

In the next step, we need to add the environment to jupyter. Use the following commands to install ipykernel and add the environment to ipykernel.

```bash
pip install ipykernel
```
```bash
python -m ipykernel install --name=myenv
```


Next, start Jupyter Notebook, download this starter notebook and open it. On the dropdown menu in the Kernel tab choose the name of the environment you created, like in the picture below.


![jupyter](https://drive.google.com/uc?export=view&id=1N-27jjlIgpTILi-_6lny7ng8sE52SAZx)


## Download and Read the Dataset

run the code below for downloading the dataset

In [4]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import missingno as msno
from sklearn.impute import SimpleImputer
from sklearn.impute import KNNImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.preprocessing import MinMaxScaler, OrdinalEncoder


def missing_table(data: pd.DataFrame):
    statistics_missing_table = data.isnull().sum().reset_index().rename(columns={"index": "Feature", 0: "CountMissing"})
    percentage_missing = (
        (statistics_missing_table["CountMissing"] / len(data) * 100).reset_index()).reset_index().rename(
        columns={"CountMissing": "PercentageMissing"})
    statistics_missing_table["PercentageMissing"] = percentage_missing["PercentageMissing"]
    statistics_missing_table["Total"] = len(data)
    return statistics_missing_table


def balance_table(data: pd.DataFrame, target_column):
    balance_table = data.groupby(target_column).size().reset_index().rename(
        columns={target_column: "Class", 0: "Count"})
    tmp = (balance_table["Count"] / len(data) * 100).reset_index().rename(
        columns={target_column: "Class", "Count": "Percentage"})
    balance_table["Percentage"] = tmp["Percentage"]
    balance_table["Total"] = len(data)
    return balance_table


def get_numerical_features_names(data: pd.DataFrame):
    result_columns = []
    for column in data.columns:
        if pd.api.types.is_any_real_numeric_dtype(data[column]):
            result_columns.append(column)

    return result_columns


def get_categorical_features_names(data: pd.DataFrame, target_feature=None):
    categorical_features = []
    for column in data.columns:
        if target_feature is not None and column == target_feature:
            continue

        if not pd.api.types.is_any_real_numeric_dtype(data[column]):
            categorical_features.append(column)
    return categorical_features


def encode_data(data: pd.DataFrame, features_to_encode: list):
    encoders = {}
    data_copy = data.copy()
    for feature in features_to_encode:
        encoder = OrdinalEncoder()
        data_copy[[feature]] = encoder.fit_transform(data_copy[[feature]])
        encoders[feature] = encoder
    return data_copy, encoders


In [5]:
!gdown 1boIax8d9Sat6OJzkiIjjpqmtSZKuRYrx

Downloading...
From: https://drive.google.com/uc?id=1boIax8d9Sat6OJzkiIjjpqmtSZKuRYrx
To: /content/ElectricCarData.csv
100% 8.20k/8.20k [00:00<00:00, 20.0MB/s]


### Import the required libraries

In [6]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

### Read the dataset

CONTEXT:
This is a dataset of electric vehicles.

It contains the following columns:


*   Brand
*   Model
*   AccelSec - Acceleration as 0-100 km/h
*   TopSpeed_KmH - The top speed in km/h
*   Range_Km - Range in km
*   Efficiency_WhKm - Efficiency Wh/km
*   FastCharge_KmH - Charge km/h
*   RapidCharge - Yes / No
*   PowerTrain - Front, rear, or all wheel drive
*   PlugType
*   BodyStyle - Basic size or style
*   Segment - Market segment
*   Seats - Number of seats
*   PriceEuro - Price in Germany before tax incentives




TASK:
Predict the target 'PriceEuro' and compare the performance of the DecisionTreeRegressor and the XGBRegressor models.

In [7]:
data = pd.read_csv('/content/ElectricCarData.csv')

In [8]:
data.head()

,Brand,Model,AccelSec,TopSpeed_KmH,Range_Km,Efficiency_WhKm,FastCharge_KmH,RapidCharge,PowerTrain,PlugType,BodyStyle,Segment,Seats,PriceEuro
0,Tesla,Model 3 Long Range Dual Motor,4.6,233,450,161,940,Yes,AWD,Type 2 CCS,Sedan,D,5,55480
1,Volkswagen,ID.3 Pure,10.0,160,270,167,250,Yes,RWD,Type 2 CCS,Hatchback,C,5,30000
2,Polestar,2,4.7,210,400,181,620,Yes,AWD,Type 2 CCS,Liftback,D,5,56440
3,BMW,iX3,6.8,180,360,206,560,Yes,RWD,Type 2 CCS,SUV,D,5,68040
4,Honda,e,9.5,145,170,168,190,Yes,RWD,Type 2 CCS,Hatchback,B,4,32997


### Encode string variables

In [9]:
missing_table(data)


,Feature,CountMissing,PercentageMissing,Total
0,Brand,0,0.0,103
1,Model,0,0.0,103
2,AccelSec,0,0.0,103
3,TopSpeed_KmH,0,0.0,103
4,Range_Km,0,0.0,103
5,Efficiency_WhKm,0,0.0,103
6,FastCharge_KmH,0,0.0,103
7,RapidCharge,0,0.0,103
8,PowerTrain,0,0.0,103
9,PlugType,0,0.0,103


## Split the dataset for training and testing in ratio 80:20

In [10]:
data_copy, encoders = encode_data(data, get_categorical_features_names(data))

In [11]:
data_copy

,Brand,Model,AccelSec,TopSpeed_KmH,Range_Km,Efficiency_WhKm,FastCharge_KmH,RapidCharge,PowerTrain,PlugType,BodyStyle,Segment,Seats,PriceEuro
0,30.0,46.0,4.6,233,450,161,50.0,1.0,0.0,2.0,7.0,3.0,5,55480
1,31.0,33.0,10.0,160,270,167,7.0,1.0,2.0,2.0,1.0,2.0,5,30000
2,23.0,0.0,4.7,210,400,181,36.0,1.0,0.0,2.0,2.0,3.0,5,56440
3,2.0,101.0,6.8,180,360,206,32.0,1.0,2.0,2.0,6.0,3.0,5,68040
4,9.0,78.0,9.5,145,170,168,2.0,1.0,2.0,2.0,1.0,1.0,4,32997
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98,20.0,6.0,7.5,160,330,191,21.0,1.0,1.0,2.0,1.0,2.0,5,45000
99,1.0,94.0,4.5,210,335,258,30.0,1.0,0.0,2.0,6.0,4.0,5,96050
100,20.0,8.0,5.9,200,325,194,21.0,1.0,0.0,2.0,1.0,2.0,5,50000
101,20.0,10.0,5.1,200,375,232,22.0,1.0,0.0,2.0,1.0,2.0,5,65000


In [12]:
X = data_copy.drop(columns="PriceEuro")
Y = data_copy["PriceEuro"]

In [13]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, shuffle=True, test_size=0.2, random_state=0)


## Initialize the DecisionTreeRegressor model, and use the fit function for training the model.

Add values for the parameters max_depth, min_samples_split, and max_features.

Fit the model using the fit function


In [14]:
data_copy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103 entries, 0 to 102
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Brand            103 non-null    float64
 1   Model            103 non-null    float64
 2   AccelSec         103 non-null    float64
 3   TopSpeed_KmH     103 non-null    int64  
 4   Range_Km         103 non-null    int64  
 5   Efficiency_WhKm  103 non-null    int64  
 6   FastCharge_KmH   103 non-null    float64
 7   RapidCharge      103 non-null    float64
 8   PowerTrain       103 non-null    float64
 9   PlugType         103 non-null    float64
 10  BodyStyle        103 non-null    float64
 11  Segment          103 non-null    float64
 12  Seats            103 non-null    int64  
 13  PriceEuro        103 non-null    int64  
dtypes: float64(9), int64(5)
memory usage: 11.4 KB


In [15]:
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor

In [16]:
decision_tree_regressor = DecisionTreeRegressor(max_depth=5, min_samples_split=20, max_features=6, random_state=0)
# decision_tree_regressor = DecisionTreeRegressor( random_state=0)

In [17]:
decision_tree_regressor.fit(x_train, y_train)

DecisionTreeRegressor(max_depth=5, max_features=6, min_samples_split=20,
                      random_state=0)

## Predict the outcomes for X test

In [18]:
tree_regressor_predict = decision_tree_regressor.predict(x_test)

## Assess the model performance, by using sklearn metrics for regression

In [19]:
from sklearn.metrics import mean_squared_error, r2_score

In [20]:
decision_tree_r2 = r2_score(y_test, tree_regressor_predict)

In [21]:
mean_squared_error(y_test, tree_regressor_predict)

872926443.8259329

## Initialize the XGBRegressor model, and use the fit function

Add values for the parameters: n_estimators, max_depth, learning_rate, and set the objective to "reg:squarederror"

Fit the model using the fit function

In [22]:
xgb_regressor = XGBRegressor(max_depth=10, n_estimators=50, learning_rate=0.1, objective='reg:squarederror')

## Predict the outcomes for X test

In [23]:
xgb_regressor.fit(x_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=10, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=50, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [24]:
xgb_regressor_predict = xgb_regressor.predict(x_test)

## Assess the model performance, by using sklearn metrics for regression

In [25]:
r2_xgboost = r2_score(y_test, xgb_regressor_predict)
r2_xgboost

0.5206246972084045

In [26]:
mean_squared_error(y_test, xgb_regressor_predict)

1219719137.4020991

## Compare the performances of both model for at least three regression metircs

In [27]:
if decision_tree_r2 >= r2_xgboost:
    print("Decision Tree better")
else:
    print("XGBoost better")

Decision Tree better
